In [0]:
import zipfile
import os

In [0]:
train_path = '/content/drive/My Drive/Train.tar.zip'
with zipfile.ZipFile(train_path, 'r') as zip:
    zip.extractall()

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import matplotlib.pyplot as plt
from torchvision import datasets

import torchvision.transforms as transforms

import numpy as np
import pandas as pd

train_on_gpu = torch.cuda.is_available()


import os
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torchvision
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision.transforms as transforms

In [0]:
#For converting the dataset to torchvision dataset format
class VowelConsonantDataset(Dataset):
    def __init__(self, file_path,train=True,transform=None):
        self.transform = transform
        self.file_path=file_path
        self.train=train
        
        self.data_info = pd.read_csv("/content/testset.csv")
        self.data_info = self.data_info.sort_values(by = ['Data'])
        self.data_info = self.data_info.reset_index()

        #print(self.data_info.head())
        #self.data_info = self.data_info.drop('index', axis =1, inplace=True)
        
        self.file_names = [file for _,_,files in os.walk(self.file_path) for file in files]
        self.file_names = sorted(self.file_names)

        self.len = len(self.file_names)
        
        
    def __len__(self):
        return len(self.file_names)
    
    def __getitem__(self,  index):
        file_name = self.file_names[index]
        image_data = self.pil_loader(self.file_path +"/" + file_name)
        if self.transform:
            image_data = self.transform(image_data)
        if self.train:
            Y1 = self.get_classes(index)
            label = Y1
            return image_data, label
            
            

    
    
    def pil_loader(self, path):
        with open(path, 'rb') as f:
            img = Image.open(f)
            image_data = img.convert('RGB')
            return image_data
    
    def get_classes(self, index):
        classs = self.data_info['Label'][index]
        return classs
    

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import matplotlib.pyplot as plt
from torchvision import datasets

import torchvision.transforms as transforms

import numpy as np
import pandas as pd

train_on_gpu = torch.cuda.is_available()

In [0]:
transform = transforms.Compose([
                            transforms.RandomResizedCrop(64),
                            transforms.ToTensor(),
                            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    
                           # mean and std in 3 channels with mean and std of 0.5 an 0.5
])



In [0]:
full_data = VowelConsonantDataset("/content/testset",train=True,transform=transform)
train_size = int(0.9 * len(full_data))
test_size = len(full_data) - train_size

train_data, validation_data = random_split(full_data, [train_size, test_size])

train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_data, batch_size=64, shuffle=True)

In [0]:
dataiter = iter(train_loader)
images, labels = dataiter.next()
labels


tensor([44, 43, 12, 40, 35,  9,  2, 15, 11, 13, 14,  4,  9, 23, 15, 44, 25, 17,
        45, 33, 33,  3, 29, 13, 28, 24, 26, 34, 37, 47, 28, 14,  6, 18, 25, 21,
         4, 20, 18,  4, 46, 22,  9,  8, 39, 20, 41, 32, 15,  0, 45, 24, 28, 12,
        43, 29, 43,  9, 40,  4, 21,  2,  9, 17])

In [0]:
print(type(images))
print(images.shape)
print(labels.shape)

<class 'torch.Tensor'>
torch.Size([64, 3, 64, 64])
torch.Size([64])


In [0]:
class Classifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(12288, 6000)
    self.fc2 = nn.Linear(6000, 1200)
    self.fc3 = nn.Linear(1200, 256)
    self.fc4 = nn.Linear(256, 128)
    self.fc5 = nn.Linear(128, 64)
    self.fc6 = nn.Linear(64,48)

  def forward(self,x):
    x = x.view(x.shape[0], -1)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    x = F.relu(self.fc4(x))
    x = F.relu(self.fc5(x))
    x = F.log_softmax(self.fc6(x), dim =1)

    return x

In [0]:
model = Classifier()

In [0]:
criterian = nn.NLLLoss()


In [0]:
optimizer = optim.Adam(model.parameters())


In [0]:
epochs = 20
for e in range(epochs):
  running_loss = 0 
  for images,labels in train_loader:
    images= images.view(images.shape[0], -1)
    optimizer.zero_grad()
    output = model.forward(images)
    #print(output)
    loss = criterian(output, labels)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
  else:
    print(f"training loss:{running_loss /len(train_loader)}")

training loss:3.8790309252562345
training loss:3.873727759608516
training loss:3.8728647108431216


KeyboardInterrupt: ignored